In [1]:
import glob, os, warnings
import logging
import datetime as dt
import logging
import os

from datetime import timedelta

logging.basicConfig(level=logging.INFO)

import datetime as dt
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation

logging.getLogger("matplotlib.font_manager").setLevel(logging.WARNING)

matplotlib.rcParams.update({
    'figure.facecolor': 'w',
    'figure.figsize': (16, 8),
    'figure.dpi': 75,
    'font.size': 12
})

import numpy as np
import pandas as pd
import cartopy.crs as ccrs

import xarray as xr 

from IPython.display import display, HTML

%matplotlib inline

In [2]:
from icenet.data.sic.mask import Masks

from icenet.plotting.forecast import process_regions
from icenet.plotting.utils import \
    filter_ds_by_obs, get_forecast_ds, get_obs_da, get_seas_forecast_da, \
    show_img, get_plot_axes

from icenet.plotting.video import xarray_to_video 
from icenet.data.sic.mask import Masks

```bash
export SCOTT_VIDEO_START="2022-4-1"
export SCOTT_VIDEO_END="$SCOTT_VIDEO_START"
./run_prediction.sh scott_video dh23 north scott_video current
./produce_op_assets.sh scott_video_north
```

In [3]:
forecast_file = "2022-04-01/2022-04-01.nc"
forecast_date = "2022-04-01"
hemisphere = "north"

masks = Masks(north=hemisphere == "north",
              south=hemisphere == "south")

fc = get_forecast_ds(forecast_file,
                     forecast_date)
obs = get_obs_da(hemisphere,
                 pd.to_datetime(forecast_date) + timedelta(days=1),
                 pd.to_datetime(forecast_date) + timedelta(days=int(fc.leadtime.max())))
fc = filter_ds_by_obs(fc, obs, forecast_date)

INFO:root:Got files: ['./data/osisaf/north/siconca/2022.nc']
INFO:root:Starting index will be 0 for 2022-04-02 00:00:00 - 2022-07-03 00:00:00


In [4]:
seas = get_seas_forecast_da(
    hemisphere,
    pd.to_datetime(forecast_date),
    bias_correct=False)

if seas is not None:
    seas = seas.assign_coords(dict(xc=seas.xc / 1e3, yc=seas.yc / 1e3))
    seas = seas.isel(time=slice(1, None))

In [5]:
masks = Masks(north=True, south=False)
land_mask = masks.get_land_mask()

In [ ]:
#fc, obs, masks = process_regions(args.region, [fc, obs, masks])

In [7]:
def update(date):
    fc_img.set_data(fc.sel(time=date))
    ob_img.set_data(obs.sel(time=date))
    
    image_title.set_text("{:04d}/{:02d}/{:02d}".format(
        date.year, date.month, date.day))

    return fc_img, ob_img, image_title

video_dates = [pd.Timestamp(date).to_pydatetime()
               for date in fc.time.values]

fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 8),)

for ax in axs:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

cmap = matplotlib.colormaps.get_cmap("viridis")
cmap.set_bad(color="dimgrey")

date = pd.Timestamp(fc.time.values[0]).to_pydatetime()
fc_img = axs[0].imshow(fc.sel(time=date),
                       cmap=cmap,
                       animated=True,
                       zorder=1)
ob_img = axs[1].imshow(obs.sel(time=date),
                       cmap=cmap,
                       animated=True,
                       zorder=1)

image_title = fig.suptitle("{:04d}/{:02d}/{:02d}".
                           format(date.year, date.month, date.day),
                           fontsize="medium", zorder=2)

axs[0].contourf(land_mask,
                levels=[.5, 1],
                colors=[cm.gray(180)],
                zorder=3)
axs[1].contourf(land_mask,
                levels=[.5, 1],
                colors=[cm.gray(180)],
                zorder=3)
#fig.colorbar(fc_img, ax=axs[0:1], shrink=0.2)

animation = FuncAnimation(fig,
                          update,
                          video_dates,
                          interval=1000/20)

plt.close()

animation.save("guess_video.mp4",
               fps=20,
               extra_args=['-vcodec', 'libx264'])

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
INFO:matplotlib.animation:figure size in inches has been adjusted from 15.0 x 8.0 to 14.986666666666666 x 8.0
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1124x600 -pix_fmt rgba -r 20 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y guess_video.mp4


In [8]:
from IPython.display import Video
Video("guess_video.mp4")